In [1]:
def new_board():
    return ((0,0,0),(0,0,0),(0,0,0))

In [4]:
def apply_move(board_state, move, side):
    move_x, move_y = move
    state_list = list(list(s) for s in board_state)
    state_list[move_x][move_y] = side
    return tuple(tuple(s) for s in state_list)

In [10]:
x= new_board()
y = apply_move(x, [0,0], 1)
y

((1, 0, 0), (0, 0, 0), (0, 0, 0))